# Script to scrap new platforms using newsAPI

Documentation: https://newsapi.org/docs

In [99]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
from urllib import request
from newsapi import NewsApiClient
from urllib.parse import urlencode

newsapi = NewsApiClient(api_key="api_key_here")

domain = "channelnewsasia.com"

In [ ]:
"""
Getting list of article urls from newsapi
"""

list_of_articles = []
for i in range(1, 6):
    print(f"working on page: {i}")
    all_articles = newsapi.get_everything(
        domains=domain, language="en", sort_by="relevancy", page=i
    )
    list_of_articles += all_articles["articles"]
list_of_urls = list(map(lambda item: item["url"], list_of_articles))
print(len(list_of_articles))
print(list_of_articles)
print(list_of_urls)

## Scraping CNA

In [ ]:
# For CNA
API_KEY = "api key here"


# Using scrapeops to bypass forbidden restriction on CNA
def get_scrapeops_url(url:str) -> str:
    """
    get the scrapops url from the article url

    Args:
        url (str): article url

    Returns:
        str: scrapops url
    """
    payload = {"api_key": API_KEY, "url": url}
    proxy_url = "https://proxy.scrapeops.io/v1/?" + urlencode(payload)
    return proxy_url


# for channelnewsasia
articles = []


# Method to get content out of the articles
def get_article_content(article_url:str) -> None:
    """
    Getting article content and storing it in the articles list

    Args:
        article_url (str): article url
    """
    r = requests.get(get_scrapeops_url(article_url))
    soup = BeautifulSoup(r.content, "html.parser")
    print(f"Working on {counter}: {article_url}")

    try:
        headline = soup.find("h1", class_="h1 h1--page-title").text.strip()

        # Getting text from all p tags
        content = map(lambda c: c.text, soup.find_all("p", class_=None))
        content_text = " ".join([str(item) for item in content])

        # Getting links
        all_links = list(
            map(
                lambda tag: tag["href"].strip(),
                soup.find_all("a", class_="h6__link list-object__heading-link"),
            )
        )

        articles.append(
            {"title": headline, "content": content_text, "links": all_links}
        )
        print("task completed")
    except Exception as e:
        print(f"Failed for: {article_url}")
        print(e)


counter = 1
for article_url in list_of_urls:
    if counter == 100:
        break
    get_article_content(article_url)
    counter += 1

# Print articles once done
print(articles)

## Scraping Straitstimes

In [ ]:
# For straits times
articles = []
for article_url in list_of_urls:
    html = request.urlopen(article_url).read().decode("utf8")
    soup = BeautifulSoup(html, "html.parser")
    print(f"Working on: {article_url}")

    try:
        # Getting headline, content and connecting url
        headline = soup.find("h1", class_="headline node-title").text.strip()

        # Getting text from all p tags
        content = map(lambda c: c.text, soup.find_all("p", class_=None))
        content_text = " ".join([str(item) for item in content])

        # Getting links
        all_links = list(map(lambda tag: tag['href'].strip(), soup.select('p a[href]')))

        articles.append({
            "title": headline, 
            "content": content_text,
            "links": all_links
        })
    except Exception as e:
        print(f"Failed for: {article_url}")
        print(e)
print(articles)

# Getting the articles in csv format

In [ ]:
print(articles)
filename = "newsapi-CNA.csv"
df = pd.DataFrame.from_records(articles)
df.to_csv(filename)